# 【問題1】コンペティション内容の確認

## コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。

# 何を学習し、何を予測するのか

 - クライアントの特徴項目を学習し、返済能力を予測する。

## どのようなファイルを作りKaggleに提出するか

- 特徴量同士の関係性を調べ、返済能力に関わりの深い項目を抜擢し、最終的な見解を加えて提出

## 提出されたものはどういった指標値で評価されるのか

- 提出物は 、予測された確率と観察されたターゲットの間のROC曲線の下の領域で評価されます。

まずは単純な方法による ベースラインモデル　を作成します。精度の基準となるモデルです。
精度を高くする必要はありません。エラーなく実行でき、Kaggle側に推定値を提出できることを目指します。

# 【問題2】学習と検証

データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。
検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

データの分析
　　目的変数に関わりの深い項目をピックアップし、検証する
  　二値分類か多値分類か選択する
前処理
　　ピックアップした項目を加工する
    訓練用にデータを分割する
学習方法を選び、必要なら標準化をする
　　検証データをメソッドを使い推定する
　　検証データの特徴量を入力して得られた推定値と、検証データの正解値の近さを計算し、学習したモデルを評価します
必要に応じて可視化する
ROC曲線を作成
　　予測された確率と観察されたターゲットの間のROC曲線の下の領域で評価される
テストデータに対して推定を行い、Kaggleに提出を行う

In [ ]:
#主要なものをインポートする
import missingno as msno
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings #ワーニング関連のモジュール？
warnings.filterwarnings('ignore') #ワーニングが消える？


df = pd.read_csv("application_train.csv")
#検証
df
#ROC曲線を描くには、真陽性率（TPR）と偽陽性率（FPR）のみが必要です
#TPRは、テスト中に利用可能なすべての陽性サンプルの中で発生する正しい陽性結果の数を定義します
#FPRは、テスト中に利用可能なすべての陰性サンプルの中で発生する誤った陽性結果の数を

## データフレームの情報

In [ ]:
#情報
df.info()

In [ ]:
#大きさ
df.shape

In [ ]:
#確認
df.head()

## 前処理

AMT_INCOME_TOTAL,AMT_CREDIT,AMT_GOODS_PRICEを比較特徴量として抜き出す。

In [ ]:
#抜き出し
df_selected = df.loc[:, ["AMT_INCOME_TOTAL", "AMT_CREDIT","TARGET"]]#, "AMT_GOODS_PRICE"
df_selected.head()

欠損値のある行を削除

In [ ]:
#削除
df_selected = df_selected.dropna(how='any')
df_selected.head()
df_selected.shape

確認

In [ ]:
#確認
missing_data = pd.DataFrame(df_selected.isnull().sum())
missing_data

## 学習モデル

In [ ]:
# sklearnのから各種モデルをimportする
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# データの前処理
# データを分割
X_train, X_test, y_train, y_test = train_test_split(df_selected.iloc[:, :-1], df_selected.loc[:, "TARGET"], test_size=0.25, random_state=42)

# データを確認
print("X_train.shape : {}, X_test.shape : {}".format(X_train.shape, X_test.shape))
print("y_train.shape : {}, y_test.shape : {}".format(y_train.shape, y_test.shape))


## 最近傍法

In [ ]:
# sklearnから標準化をするモデルをインポートする
from sklearn.preprocessing import StandardScaler
# モデルをインスタンス化して、モデルの中にあるメソッドを使える状態にする。
scaler = StandardScaler()

# 標準化をする（訓練用データで.fitを行う）
scaler.fit(X_train)

# 標準化をする（訓練用、検証用双方で.transformを行う）
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [ ]:
# sklearnから最近傍法をするモデルをインポートする
from sklearn.neighbors import KNeighborsClassifier

# モデルに引数を渡してインスタン化する
neigh_1 = KNeighborsClassifier(n_neighbors=1)
neigh_3 = KNeighborsClassifier(n_neighbors=3)
neigh_5 = KNeighborsClassifier(n_neighbors=5)

# 学習する（訓練用データを.fitする）
neigh_1.fit(X_train_transformed, y_train)
neigh_3.fit(X_train_transformed, y_train)
neigh_5.fit(X_train_transformed, y_train)

# 推定する（検証用データを.predictする）
y_pred_neigh_1 = neigh_1.predict(X_test_transformed)
y_pred_neigh_3 = neigh_3.predict(X_test_transformed)
y_pred_neigh_5 = neigh_5.predict(X_test_transformed)

In [ ]:
# モデルmetrics をimportすればaccuracy等を一つずつインポートしなくても良い
from sklearn import metrics 

# 評価する関数を作成する
def evaluate(y_true, y_pred):
    """
    2値分類の評価指標を計算する
    """
    acc = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    confusion = metrics.confusion_matrix(y_test, y_pred)
    return acc, precision, recall, f1, confusion

print("1-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_1)))
print("------")

print("3-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_3)))
print("------")

print("5-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_5)))

# 【問題3】テストデータに対する推定

In [ ]:
df_1 = pd.read_csv("application_test.csv")

df_1

In [ ]:
#テストデータ抜き出し
df_selected_1 = df_1.loc[:, ["AMT_INCOME_TOTAL", "AMT_CREDIT"]]#AMT_GOODS_PRICE
df_selected_1.head()

In [ ]:
y_pred_neigh_5 = neigh_5.predict(df_selected_1)
y_pred_neigh_5_proba = neigh_5.predict_proba(df_selected_1)
print(y_pred_neigh_5_proba)
print(y_pred_neigh_5_proba.shape)
print(type(y_pred_neigh_5_proba))

In [ ]:
answer = y_pred_neigh_5_proba[:,1]
print(answer)

In [ ]:
df_selected_2 = df_1.loc[:, ["SK_ID_CURR"]]  #IDのカラムを抽出
df_selected_3 = pd.Series(answer) #NP配列をDFに返還
df_answer =pd.concat([df_selected_2,df_selected_3],axis = 1) #DF同士を結合
print(df_answer)

In [ ]:
df_answer_1 = df_answer.rename(columns ={0:'TARGET'}) #0のカラムをTARGETに変更
print(df_answer_1)

In [ ]:
df_answer_1.to_csv('/Users/daisukesatou/diveintocode-ml/dic_sinyo.csv',index=False)
#INDEXを除いたDFをCSVに書き込む

In [ ]:
# 落とし込むカレントを検索
import os

path = os.getcwd()

print(path)       

- # Kaggleに提出、エラー無し

# 【問題4】特徴量エンジニアリング
## 精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。
  - どの特徴量を使うか
  - どう前処理をするか
何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。

In [ ]:
# 引数の値を変えてみる
y_pred_neigh_7 = neigh_3.predict(df_selected_1)
y_pred_neigh_7_proba = neigh_3.predict_proba(df_selected_1)

In [ ]:
print(y_pred_neigh_7_proba)
print(y_pred_neigh_7_proba.shape)
print(type(y_pred_neigh_7_proba))

In [ ]:
#最近傍方の引数を変えてみる
answer_1 = y_pred_neigh_7_proba[:,1]
print(answer_1)

In [ ]:
#問３と同じ処理
df_selected_2 = df_1.loc[:, ["SK_ID_CURR"]]
df_selected_3 = pd.Series(answer)
df_answer =pd.concat([df_selected_2,df_selected_3],axis = 1)
print(df_answer_1)

-  # 引数の値を変えてみたが特に変化は無し

## 特徴量を変えてみる。

In [ ]:
#"AMT_CREDIT"を"AMT_GOODS_PRICE"に変更
df_selected_2 = df_1.loc[:, ["AMT_INCOME_TOTAL","AMT_GOODS_PRICE"]]
df_selected_2.head()

In [ ]:
# 問３と同じ処理
y_pred_neigh_6 = neigh_5.predict(df_selected_2)
y_pred_neigh_6_proba = neigh_5.predict_proba(df_selected_2)
print(y_pred_neigh_6_proba)
print(y_pred_neigh_6_proba.shape)
print(type(y_pred_neigh_6_proba))

In [ ]:
answer_1= y_pred_neigh_6_proba[:,1]
print(answer_1)

In [ ]:
df_selected_4 = pd.Series(answer_1)
df_selected_3 = df_1.loc[:, ["SK_ID_CURR"]]
df_answer_2 =pd.concat([df_selected_3,df_selected_4],axis = 1)
df_answer_3 = df_answer_2.rename(columns ={0:'TARGET'})
print(df_answer_3)

-  # 特徴量を変えてみたが、ほぼ変化なし。

# 前処理の変更

## 学習方法を変えてみる

In [ ]:
# データの前処理
# データを分割
X_train, X_test, y_train, y_test = train_test_split(df_selected.iloc[:, :-1], df_selected.loc[:, "TARGET"], test_size=0.25, random_state=42)

# データを確認
print("X_train.shape : {}, X_test.shape : {}".format(X_train.shape, X_test.shape))
print("y_train.shape : {}, y_test.shape : {}".format(y_train.shape, y_test.shape))


In [ ]:
# importしたモデルをインスタンス化
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

lr = LogisticRegression()
svc = SVC()
tree = DecisionTreeClassifier()
r_forest_c = RandomForestClassifier()

In [ ]:
# 評価する関数を作成する
def evaluate(y_true, y_pred):
    """
    2値分類の評価指標を計算する
    """
    acc = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    confusion = metrics.confusion_matrix(y_test, y_pred)
    return acc, precision, recall, f1, confusion


In [ ]:
#ロジスティック回帰
print("Logistic Regression")
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred)))

In [ ]:
#検証
y_pred_lr = lr.predict(df_selected_2)
y_pred_lr_proba = lr.predict_proba(df_selected_2)
answer_2= y_pred_lr_proba[:,1]
print(answer_2)

- # ロジスティック回帰による学習ではわずかながら変化があった。

In [ ]:
#決定木
print("Decision Tree")
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred)))

In [ ]:
y_pred_tree = tree.predict(df_selected_2)
y_pred_tree_proba = tree.predict_proba(df_selected_2)
answer_3= y_pred_tree_proba[:,1]
print(answer_3)

- # 決定木による学習でもわずかながら変化があった。

In [ ]:
#ランダムフォレスト
print("Random Forest")
r_forest_c.fit(X_train, y_train)
y_pred = r_forest_c.predict(X_test)
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred)))

In [ ]:
y_pred_forest = r_forest_c.predict(df_selected_2)
y_pred_forest_proba = r_forest_c.predict_proba(df_selected_2)
answer_4= y_pred_forest_proba[:,1]
print(answer_4)

- # ランダムフォレストの結果をみても微細ながら変化は出た。

## 以上　５項目を検証した結果になります。